In [1]:
# Importing Useful Libraries

import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time
import copy

In [2]:
# Function to create an adjacency matrix

def Amat(size,num):
    matrix = np.zeros((size,size))
    count=0
    while count!=num:
        x,y=np.random.randint(size),np.random.randint(size)
        if matrix[x][y]==0 and x != y:
            matrix[x][y] = matrix[y][x]=1
            count = count+1
    return matrix

# Function to assign randomly straties to individual of population denoted by indices

def Population(Size, Frac):
    Pop = np.zeros(Size)
    for i in range(int(Frac*Size)):
        Pop[i] = 1
    rd.shuffle(Pop)
    return Pop

In [3]:
# Function to collect payoff for each indivial in the population after interaction

def PayOff(population, Amatrix, Mmatrix):
    PO = np.zeros(len(population))
    for i in range(len(population)):
        p = 0
        for j in range(len(population)):
            if   Amatrix[i][j] == 1 and population[i] == 0 and population[j] == 0:
                p += Mmatrix[0]
                
            elif Amatrix[i][j] == 1 and population[i] == 0 and population[j] == 1:
                p += Mmatrix[1]
                
            elif Amatrix[i][j] == 1 and population[i] == 1 and population[j] == 0:
                p += Mmatrix[2]
                
            elif Amatrix[i][j] == 1 and population[i] == 1 and population[j] == 1:
                p += Mmatrix[3]
        PO[i] = p
                
    return np.array(PO)

In [4]:
# Function to obtain probability of changing strategy

def PIE(i, j, S, T, Amatrix, Pmatrix):
    Ki = np.sum(Amatrix[i])
    Kj = np.sum(Amatrix[j])
    Numer = Pmatrix[j] - Pmatrix[i]
    Denom = max(Ki, Kj)*( max(1, T) - min(0, S) )
    return Numer/Denom

In [5]:
# Function to do one round of simulation for a given population, network and payoff matrix

def Simulation(Iter, population, Amatrix, Mmatrix, DP):
    Copy_Pop = copy.deepcopy(population)
    FreqD = np.zeros(Iter+1) # Array to collect frequency of defector after each generation
    FreqD[0] = (np.sum(Copy_Pop)/len(Copy_Pop))
    
    for n in range(Iter):
        
        Pmatrix = PayOff(Copy_Pop, Amatrix, Mmatrix) # Collecting Payoff
        
        for i in range(len(Copy_Pop)):
            Neighbr = np.where(Amatrix[i] == 1)
            
            try:
                jr = rd.choice(Neighbr[0])
                if Pmatrix[i] < Pmatrix[jr] and Copy_Pop[i] != Copy_Pop[jr]  and rd.random() < PIE(i, jr, Mmatrix[1], Mmatrix[2], Amatrix, Pmatrix):
                    Copy_Pop[i] = Copy_Pop[jr]
            except:
                None
                
        print(DP,":" ,"%3.3f" %((n/Iter)*100), '%', end="\r") # To display progress of simulation
        time.sleep(10**(-4))
        
        FreqD[n+1] = (np.sum(Copy_Pop)/len(Copy_Pop)) # Appending frequencies to np array
    return Copy_Pop, np.array(FreqD)

In [6]:
# Creating Population and Adjacency Matrices

Popu1 = Population(200, 0.5)
A0 = Amat(200, 600)

In [7]:
# Function to create values of pair of value S and T

def ST_value(N1):
    T1 = np.linspace(0, 2, N1)
    S1 = np.linspace(-1, 1, N1)
    Y1 = []
    for i in range(N1):
        Z1 = []
        for j in range(N1):
            X1 = [S1[i], T1[j]]
            Z1.append(X1)
        Y1.append(Z1)
    return np.array(Y1)

In [9]:
# Function to run the simulation for different different payoff matrices

def Sim_Loop(NN, Popu, Amatrix, T0, TG):
    YY = ST_value(NN)
    ZZ = []
    for s in range(NN):
        TD = []
        for t in range(NN):
            Pick = YY[s][t]
            MM = np.array([1, Pick[0], Pick[1], 0]) # Creating new payoff matrix
            PP, FF = Simulation(T0 + TG, Popu, Amatrix, MM, (s, t)) # Doing Simulation
            TD.append(FF)
        np.savetxt('Data_S'+str(s)+'.csv' , TD, delimiter=',') # Exporting defector frequencies to a csv file for further analysis 
        ZZ.append(TD)
    print('Done Duna Done Done')
    return np.array(ZZ)

In [10]:
# Running the Simulation
Result1 = Sim_Loop(31, Popu1, A0, 2000, 500)